# 策略持仓权重管理
---

In [ ]:
# 设置 Redis 链接信息，用于存储持仓权重信息
import os
os.environ['RWC_REDIS_URL'] = 'redis://***:***@***:6379/0'

# 也可以使用 dotenv 来设置环境变量
from dotenv import load_dotenv
# load_dotenv(r'path/to/.env', override=True)
load_dotenv(r"D:\ZB\git_repo\waditu\czsc\examples\test_offline\.env", override=True)

import czsc
import redis
import pandas as pd

# print("redis_url", os.getenv("RWC_REDIS_URL"))

### 一、写入策略持仓权重


策略持仓权重样例数据可以从飞书文档中下载：

In [2]:
# 创建 BlockingConnectionPool 连接池，必须设置 decode_responses=True 参数，否则返回的数据是 bytes 类型
connection_pool = redis.BlockingConnectionPool.from_url(os.getenv("RWC_REDIS_URL"), decode_responses=True)

# 策略写入阶段，必须设置 send_heartbeat=True 参数，这样可以在写入数据时自动更新策略的最新写入时间
rwc = czsc.RedisWeightsClient('TestStrategy', connection_pool=connection_pool, key_prefix='Weights', send_heartbeat=True)

# 首次写入，建议设置一些策略元数据
rwc.set_metadata(description='测试策略：仅用于读写redis测试', base_freq='日线', author='测试A', outsample_sdt='20220101')

# 查看策略元数据
print("策略元数据：", rwc.metadata)

# 写入策略持仓权重，样例数据下载：https://s0cqcxuy3p.feishu.cn/wiki/Pf1fw1woQi4iJikbKJmcYToznxb
dfw = pd.read_feather(r"C:\Users\zengb\Downloads\weight_example.feather")
print("持仓权重数据：\n", dfw.head())

# on_bar 模式下按单条数据写入，效率更高，可以实时更新持仓权重
# rwc.publish(**dfw.iloc[0].to_dict())

# 盘后或者定时执行的策略，批量写入可以提高效率
rwc.publish_dataframe(dfw, overwrite=False, batch_size=10000)

2024-03-09 20:17:45.154 | INFO     | czsc.traders.rwc:__init__:62 - TestStrategy Weights: 使用传入的 redis 连接池


策略元数据： {'name': 'TestStrategy', 'base_freq': '日线', 'key_prefix': 'Weights', 'description': '测试策略：仅用于读写redis测试', 'author': '测试A', 'outsample_sdt': '20220101', 'update_time': '2024-03-09 20:17:47', 'kwargs': '{}'}


2024-03-09 20:17:48.569 | INFO     | czsc.traders.rwc:publish_dataframe:172 - 输入数据中有 3938513 条权重信号


持仓权重数据：
                    dt    symbol  weight  price
0 2018-03-26 09:01:00  SEsc9001     0.0  442.9
1 2018-03-26 09:02:00  SEsc9001     0.0  441.2
2 2018-03-26 09:03:00  SEsc9001     0.0  440.4
3 2018-03-26 09:04:00  SEsc9001     0.0  439.5
4 2018-03-26 09:05:00  SEsc9001     0.0  440.7


2024-03-09 20:17:49.073 | INFO     | czsc.traders.rwc:publish_dataframe:182 - 去除单个品种下相邻时间权重相同的数据后，剩余 29906 条权重信号
2024-03-09 20:17:49.298 | INFO     | czsc.traders.rwc:publish_dataframe:199 - 不允许重复写入，已过滤 0 条重复信号
2024-03-09 20:17:49.433 | INFO     | czsc.traders.rwc:publish_dataframe:223 - 索引 0，即将发布 10000 条权重信号
2024-03-09 20:17:54.251 | INFO     | czsc.traders.rwc:publish_dataframe:225 - 已完成 10000 次发布
2024-03-09 20:17:54.252 | INFO     | czsc.traders.rwc:publish_dataframe:223 - 索引 10000，即将发布 10000 条权重信号
2024-03-09 20:17:57.557 | INFO     | czsc.traders.rwc:publish_dataframe:225 - 已完成 20000 次发布
2024-03-09 20:17:57.559 | INFO     | czsc.traders.rwc:publish_dataframe:223 - 索引 20000，即将发布 9906 条权重信号
2024-03-09 20:17:59.983 | INFO     | czsc.traders.rwc:publish_dataframe:225 - 已完成 29905 次发布
2024-03-09 20:18:00.196 | INFO     | czsc.traders.rwc:update_last:105 - 更新 Weights:LAST:TestStrategy 的 last 时间


29905

### 二、读取策略持仓权重



In [4]:
# 获取 redis 中的策略元数据
dfm = czsc.get_strategy_mates(key_pattern='Weights:META:*')
dfm

,name,base_freq,key_prefix,description,author,outsample_sdt,update_time,kwargs,heartbeat_time
0,TestStrategy,日线,Weights,测试策略：仅用于读写redis测试,测试A,20220101,2024-03-09 20:17:47,{},2024-03-09 20:19:18


In [6]:
# 获取 redis 中的指定策略的持仓权重（全部品种，全部历史）
dfw = czsc.get_strategy_weights(strategy_name='TestStrategy', connection_pool=connection_pool, key_prefix='Weights')
dfw

2024-03-09 20:20:25.875 | INFO     | czsc.traders.rwc:__init__:62 - TestStrategy Weights: 使用传入的 redis 连接池


,dt,symbol,weight,update_time
0,2017-01-03 09:01:00,DLeb9001,0.000000,2024-03-09 20:17:49
1,2017-01-03 09:01:00,DLeg9001,0.000000,2024-03-09 20:17:49
2,2017-01-03 09:01:00,DLj9001,0.000000,2024-03-09 20:17:49
3,2017-01-03 09:01:00,SEsc9001,0.000000,2024-03-09 20:17:49
4,2017-01-03 09:01:00,SQag9001,0.000000,2024-03-09 20:17:49
...,...,...,...,...
229558,2023-07-31 14:36:00,SQag9001,-0.705882,2024-03-09 20:17:49
229559,2023-07-31 14:36:00,ZZAP9001,0.954545,2024-03-09 20:17:49
229560,2023-07-31 14:36:00,ZZSA9001,0.277778,2024-03-09 20:17:49
229561,2023-07-31 14:36:00,ZZSF9001,0.545455,2024-03-09 20:17:49


In [7]:
# 获取 redis 中的指定策略的持仓权重（指定品种全部历史）
dfw = czsc.get_strategy_weights(strategy_name='TestStrategy', symbols=['DLeb9001', 'ZZAP9001'])
dfw

2024-03-09 20:20:41.617 | INFO     | czsc.traders.rwc:__init__:67 - TestStrategy Weights: 使用环境变量 RWC_REDIS_URL 创建 redis 连接池


,dt,symbol,weight,update_time
0,2017-01-03 09:01:00,DLeb9001,0.000000,2024-03-09 20:17:49
1,2017-01-03 09:01:00,ZZAP9001,0.000000,2024-03-09 20:17:49
2,2017-01-03 09:06:00,DLeb9001,0.000000,2024-03-09 20:17:49
3,2017-01-03 09:06:00,ZZAP9001,0.000000,2024-03-09 20:17:49
4,2017-01-03 13:45:00,DLeb9001,0.000000,2024-03-09 20:17:49
...,...,...,...,...
51009,2023-07-31 13:36:00,ZZAP9001,0.954545,2024-03-09 20:17:49
51010,2023-07-31 14:31:00,DLeb9001,0.954545,2024-03-09 20:17:49
51011,2023-07-31 14:31:00,ZZAP9001,0.954545,2024-03-09 20:17:49
51012,2023-07-31 14:36:00,DLeb9001,0.954545,2024-03-09 20:17:49


In [8]:
# 获取 redis 中的指定策略的持仓权重（指定时间段）
dfw = czsc.get_strategy_weights(strategy_name='TestStrategy', symbols=['DLeb9001', 'ZZAP9001'], sdt='20230101', edt='20240131')
dfw = dfw.sort_values(['symbol', 'dt'])
dfw

2024-03-09 20:21:00.916 | INFO     | czsc.traders.rwc:__init__:67 - TestStrategy Weights: 使用环境变量 RWC_REDIS_URL 创建 redis 连接池


,dt,symbol,weight,update_time
0,2023-01-03 09:06:00,DLeb9001,0.954545,2024-03-09 20:17:49
2,2023-01-03 09:11:00,DLeb9001,0.954545,2024-03-09 20:17:49
4,2023-01-03 09:15:00,DLeb9001,0.954545,2024-03-09 20:17:49
6,2023-01-03 09:21:00,DLeb9001,0.954545,2024-03-09 20:17:49
8,2023-01-03 09:26:00,DLeb9001,0.954545,2024-03-09 20:17:49
...,...,...,...,...
4705,2023-07-31 11:01:00,ZZAP9001,0.954545,2024-03-09 20:17:49
4707,2023-07-31 11:26:00,ZZAP9001,0.954545,2024-03-09 20:17:49
4709,2023-07-31 13:36:00,ZZAP9001,0.954545,2024-03-09 20:17:49
4711,2023-07-31 14:31:00,ZZAP9001,0.954545,2024-03-09 20:17:49


In [9]:
# 获取 redis 中的指定策略的最后一个时刻的持仓权重
dfw = czsc.get_strategy_weights(strategy_name='TestStrategy', only_last=True)
dfw

2024-03-09 20:21:17.413 | INFO     | czsc.traders.rwc:__init__:67 - TestStrategy Weights: 使用环境变量 RWC_REDIS_URL 创建 redis 连接池


,symbol,weight,dt,update_time,price,ref
0,ZZAP9001,0.954545,2023-07-27 09:11:00,2024-03-09 20:17:49,9701.066217704249,{}
1,ZZUR9001,0.400000,2023-07-28 10:36:00,2024-03-09 20:17:49,3535.4438227861174,{}
2,DLeb9001,0.954545,2023-07-28 14:16:00,2024-03-09 20:17:49,10144.286683491264,{}
3,SEsc9001,1.000000,2023-07-29 02:16:00,2024-03-09 20:17:49,415.3633199230844,{}
4,ZZSA9001,0.277778,2023-07-31 09:06:00,2024-03-09 20:17:49,1731.4345038436766,{}
5,DLj9001,-0.636364,2023-07-31 11:01:00,2024-03-09 20:17:49,3949.359200482296,{}
6,DLeg9001,0.894737,2023-07-31 14:31:00,2024-03-09 20:17:49,3586.740974496101,{}
7,ZZSF9001,0.545455,2023-07-31 14:31:00,2024-03-09 20:17:49,10657.215688529335,{}
8,SQag9001,-0.705882,2023-07-31 14:36:00,2024-03-09 20:17:49,3950.9596447677372,{}


### 三、删除 redis 中的策略

In [12]:
import os
import czsc

# 获取 RWC_REDIS_URL 环境变量，用于连接 Redis
# redis_url = os.getenv("RWC_REDIS_URL")
# czsc.clear_strategy(strategy_name="TestStrategy", redis_url=redis_url, key_prefix="Weights")

# redis_url 参数可以省略，如果省略则使用环境变量 RWC_REDIS_URL
czsc.clear_strategy(strategy_name="TestStrategy", key_prefix="Weights", connection_pool=connection_pool)

2024-03-09 20:24:03.874 | INFO     | czsc.traders.rwc:__init__:62 - TestStrategy Weights: 使用传入的 redis 连接池
2024-03-09 20:24:11.224 | INFO     | czsc.traders.rwc:clear_all:262 - TestStrategy 删除了 29926 条记录
